In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
data

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高
5,TVciHBPL5XmUxMEd,我开始就是荣放2.5 森林人2.5二选一 荣放主要是底盘质感不行 太硬 其次是...,价格,1,便宜
6,4lb9TsO5rjqUy1Bu,唉，这货的价格死硬死硬的，低配版优惠1万据说已经罕有了。,价格,-1,死硬
7,z98xV7MphZIF4EvO,价格的话只能说一般般吧，太仓前段时间定的比你便宜！,价格,0,一般
8,hoWrMnGTUvdsK3Dm,听过，价格太贵，但一直念念不忘,价格,-1,贵
9,r1WHgbI0ZKqMdELy,恭喜恭喜，这个优惠不错哦！,价格,1,优惠不错


In [3]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
len(data['content_id'].unique()), len(list(data['content_id']))
content_ids = data['content_id'].unique()
subjs = data['subject'].unique()
df = {}
text, subs, sentis = [], [], []
for i in range(len(content_ids)):
    if i % 100 == 0:print(i)
    dd = data[data['content_id'] == content_ids[i]]
    text.append(dd.loc[dd.index.tolist()[0], 'content'])
    temp_subs, temp_senti = [], []
    for sub in subjs:
        temp_subs.append(sub)
        if len(dd[dd['subject'] == sub]) != 0:
            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))
        else:
            temp_senti.append('2')
    subs.append(','.join(temp_subs))
    sentis.append(','.join(temp_senti))
df['content'] = text
df['subjects'] = subs
df['sentiments'] = sentis
df = pd.DataFrame(df)
df.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)
#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nlen(data['content_id'].unique()), len(list(data['content_id']))\ncontent_ids = data['content_id'].unique()\nsubjs = data['subject'].unique()\ndf = {}\ntext, subs, sentis = [], [], []\nfor i in range(len(content_ids)):\n    if i % 100 == 0:print(i)\n    dd = data[data['content_id'] == content_ids[i]]\n    text.append(dd.loc[dd.index.tolist()[0], 'content'])\n    temp_subs, temp_senti = [], []\n    for sub in subjs:\n        temp_subs.append(sub)\n        if len(dd[dd['subject'] == sub]) != 0:\n            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))\n        else:\n            temp_senti.append('2')\n    subs.append(','.join(temp_subs))\n    sentis.append(','.join(temp_senti))\ndf['content'] = text\ndf['subjects'] = subs\ndf['sentiments'] = sentis\ndf = pd.DataFrame(df)\ndf.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)\n#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')\n"

In [4]:
data = pd.read_csv('../../../data/df_sen_sub/merge_train.csv')
data = data.fillna('')
subs = list(data['subjects'])
sentiments = list(data['sentiments'])
content = list(data['content'])
test_data = pd.read_csv('../../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
data = data.fillna('')
senti_words = list(data['sentiment_word'])

In [5]:
with open('../../../data/df_sen_sub/add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs) + senti_words))
# 迭代器，使用jieba将句子进行分词
stopwords = open('../stopwords.txt', 'r').read().split('\n')
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                res = []
                for word in list(jieba.cut(line)):
                    if word in stopwords:
                        continue
                    res.append(word)
                yield res#list(jieba.cut(line))

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    #model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    #model.save("../../data/df_sen_sub/add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../../data/df_sen_sub/add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../../data/df_sen_sub/add_word2vec_model")
sentences = list(sentences)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.960 seconds.
Prefix dict has been built succesfully.


In [6]:
len(sentences)

28890

In [7]:
from collections import Counter
data = pd.read_csv('../../../data/df_sen_sub/train.csv')
sub_uniq = data['subject'].unique()
data = data[pd.notnull(data['sentiment_word'])]
senti_word_set = {}
sum_ = 0
for key in sub_uniq:
    co = 0
    senti_word_set[key] = {}
    #dd = list(data[data['subject'] == key]['sentiment_word'])
    ddd = data[data['subject'] == key]
    for kk in [-1, 0, 1]:
        dd = list(ddd[ddd['sentiment_value'] == kk]['sentiment_word'])
        ss = []
        for phras in dd:
            words = list(jieba.cut(phras))
            for word in words:
                word = word.replace(' ', '')
                if word == '' or word in stopwords:continue
                ss.append(word)
                co += 1
            counter = Counter(ss)
            sort = sorted(counter.items(), key=lambda item: item[1], reverse = True)
            for mm in counter:
                if mm not in senti_word_set[key]:
                    senti_word_set[key][mm] = kk
                elif senti_word_set[key][mm] == kk:
                    continue
                else:
                    senti_word_set[key][mm] = 0
        sum_ += co
print(sum_)
senti_word_set

6431


{'价格': {'高': 0,
  '有钱': -1,
  '任性': -1,
  '死硬': -1,
  '贵': 0,
  '优惠': 0,
  '太小': -1,
  '套路': -1,
  '太贵': -1,
  '不值': -1,
  '高昂': -1,
  '下不来': -1,
  '不会': 0,
  '便宜': 0,
  '放弃': -1,
  '平易近人': -1,
  '价格': 0,
  '老贵': -1,
  '没有': 0,
  '保证': -1,
  '好多': -1,
  '更': -1,
  '合适': 0,
  '那点': -1,
  '贵太多': -1,
  '挺高': -1,
  '不菲': -1,
  '一点': 0,
  '奇高': -1,
  '不好': -1,
  '小贵': 0,
  '过高': -1,
  '售价': -1,
  '明显': -1,
  '高于': -1,
  '竞品': -1,
  '优势': -1,
  '虚高': -1,
  '不高': 0,
  '性价比': 0,
  '不算': -1,
  '单价': -1,
  '没': 0,
  '降价': -1,
  '不要': 0,
  '想': -1,
  '保值': -1,
  '率': -1,
  '大众': -1,
  '好': 0,
  '略贵': -1,
  '保养': -1,
  '不用': -1,
  '考虑': -1,
  '不靠': -1,
  '谱': -1,
  '没法': -1,
  '咬咬牙': -1,
  '不合算': -1,
  '影响': 0,
  '亲民点': 0,
  '差不多': 0,
  '差距': 0,
  '知道': 0,
  '合理': 0,
  '就行': 0,
  '行情': 0,
  '价': 0,
  '正常': 0,
  '几百': 0,
  '参差不齐': 0,
  '行': 0,
  '买': 0,
  '价差': 0,
  '很大': 0,
  '低': 0,
  '不能': 0,
  '在意': 0,
  '还行': 0,
  '不错': 1,
  '幅度': 1,
  '不小': 1,
  '后悔': 1,
  '值': 1,
  '愉快': 1,
  '不够': 1,
  '比较'

In [8]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(test_content) + len(content)):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    else:
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])     
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(8290, 50, 100) (2364, 50, 100)


In [9]:
x_sent_vecs = {}
x_test_sent_vecs = {}
for key in senti_word_set:
    x_sent_vecs[key] = []
    x_test_sent_vecs[key] = []
    for i in range(len(content) + len(test_content)):
        if i < len(content):
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_sent_vecs[key].append(t_sent)
        else:
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_test_sent_vecs[key].append(t_sent)    

In [10]:
#va, content_vecs, y
pretrained.append([0.0] * 100)
word2ind['null'] = 17387
ind2word = {}
for key in word2ind:
    ind2word[word2ind[key]] = key
map_ = ['-1', '0', '1', '2']
sub_set = {}
for i in range(len(subs)):
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        if sentis[j] == '2':
            sub_set[subjs[j]].append(0)
        else:
            sub_set[subjs[j]].append(1)
        #sub_set[subjs[j]].append(map_.index(sentis[j]))

In [11]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [13]:
data = pd.read_csv('../../../data/df_sen_sub/merge_train.csv')
s = data.loc[0, 'subjects'].split(',')
for i in range(len(s)):
    print(i, s[i])

0 价格
1 配置
2 操控
3 舒适性
4 油耗
5 动力
6 内饰
7 安全性
8 空间
9 外观


In [14]:
data = pd.read_csv('../../../data/df_sen_sub/merge_train.csv')
set_ind = []
for i in range(len(data)):
    senti = int(str(data.loc[i,'sentiments']).split(',')[5]) #:动力
    if senti != 2:
        set_ind.append(i)

In [15]:
x_vecs = []
x_test_vecs = []  
ind = 0
senti_sen = []
senti_test_sen = []
for i in range(len(content)):
    if i not in set_ind:
        continue
    temp = []
    t_s = []
    for j in range(len(sentences[i])):
        if sentences[i][j] not in word2ind:
            ind += 1
        t_s.append(sentences[i][j])
        temp.append(model[sentences[i][j]])
    while len(temp) < 50:
        temp.append([0.0] * 100)
    if len(temp) > 50:
        temp = temp[:50]
    x_vecs.append(temp)
    senti_sen.append(t_s[:50])
x_vecs = np.array(x_vecs)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [16]:
x_sent_vecs = {}
for key in senti_word_set:
    x_sent_vecs[key] = []
    for i in range(len(content)):
        if i not in set_ind:
            continue
        t_sent = []
        for j in range(len(sentences[i])):
            if sentences[i][j] in senti_word_set[key]:
                t_sent.append(senti_word_set[key][sentences[i][j]])
            else:
                t_sent.append(0) 
        while len(t_sent) < 50:
            t_sent.append(0)
        if len(t_sent) > 50:
            t_sent = t_sent[:50]
        x_sent_vecs[key].append(t_sent) 

In [17]:
map_ = ['-1', '0', '1']
sub_set = {}
for i in range(len(subs)):
    if i not in set_ind:
        continue
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] != '动力':continue
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        sub_set[subjs[j]].append(map_.index(sentis[j]))

In [18]:
han_x = []
print(len(senti_sen))
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)
print(han_x.shape)

2732
(2732, 50)


In [19]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F

In [21]:
X_train, X_test, y_train, y_test = train_test_split(han_x,sub_set['动力'], test_size=0.2, random_state=7)

In [22]:
real_text = []
for i in range(len(X_test)):
    temp = []
    for j in range(len(X_test[i])):
        temp.append(ind2word[X_test[i][j]])
    real_text.append(''.join(temp))

In [23]:
embeddings = nn.Embedding(len(pretrained), 100)
pretrained_weight = np.array(pretrained)
embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
han_x = embeddings(torch.from_numpy(han_x))

In [24]:
han_x = han_x.detach().numpy()
han_x.shape

(2732, 50, 100)

In [25]:
han_x = han_x.reshape(han_x.shape[0], -1)
han_x.shape

(2732, 5000)

In [26]:
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(han_x,sub_set['动力'], test_size=0.2, random_state=seed)

In [27]:
clf = XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.75%


In [28]:
from sklearn.externals import joblib #jbolib模块

#保存Model(注:save文件夹要预先建立，否则会报错)
joblib.dump(clf, '动力.pkl')

['动力.pkl']

In [29]:
res = predictions
ll = y_test
'''
for zz in range(len(res)):
    set_ = {'价格合适','便宜','实在','价格便','价格不贵','价格不高','价格不多','价钱不贵','价钱不高','价钱不多','良心','性价比高'}
    for k in set_:
        if k in real_text[zz]:
            res[zz] = 2
            #print(real_text[zz].split('null')[0], res[zz], ll[zz])
            break
    set_ = {'小贵','更贵','价格不合适','价格贵','价钱贵','价格高','价钱高','价钱多','不值','不合适','性价比低','性价比不高'}
    for k in set_:
        if k in real_text[zz]:
            res[zz] = 0
            #print(real_text[zz].split('null')[0], res[zz], ll[zz])
            break
'''
correct, total = 0, 0
for h in range(len(res)):
    total += 1
    if res[h] == ll[h]:
        correct += 1
    else:
        print(real_text[h].split('null')[0], res[h] - 1, ll[h] - 1)

print('after change result of accuracy is : %d %%' % (100 * correct / total))

翼虎动力配置完胜森林尤其动力一点自驾游帮助翼虎空间不行前后排不行坐久舒服视野不行根本不像SUV样子觉得自驾游2.5森林更好2.0动力日常够用长途自驾弱点断轴不要年前事早就不断 1 0
大人体重平均150高速正常行驶行驶缓慢时油耗确实不高 0 1
车绝对好开耐性怀疑说漏一个发动机漏油经常油渍 0 -1
森林动力确实2.5-100加速难跑进10秒森林这车好开转向手感车神跟随性方面 1 0
森林四驱优点经济性加上较大离地间隙更好性越野性能Q不及森林 0 -1
正在考虑之中想要性能高车一部森林我意 0 1
不带分动箱斯巴鲁全系全时四驱傲虎森林一代四驱系统普通人包括反正开不出极限没事去试家用车极限反应速度应该级别森林最快路虎神行者关注新车奇骏森林人配鹰眼需要下一代最近改款新奇骏辅助安全系统非常一个女司机神器（360全景摄像头 0 1
说明书推荐使用0w20机油水平发动机机油流动性不好不能充分润滑缸体上部长期容易磨损缸体引起烧机油 0 -1
cvt反应慢半拍造成油门原因起步踩油cvt改变传动比延迟四驱起步会优先动力传给后轮传动轴一个延迟反应踩油走条件反射没踩够踩车动油门会感觉窜没办法物理瓶颈4s改不了试试踩油不动车动踩悠着点开 0 -1
简单烧机油问题轮滑系统一定故障 0 -1
习惯就行了森林买发动机加四驱辣鸡不要要求太高先摸摸纸壳内饰板看看一块布挡方向盘最后油门刹车踏板往上瞅瞅懂说 0 -1
恭喜朋友冠军版当时着迷没来动力操控应该不错 0 1
推荐昂科威2.0T精英版森林价格高保值率低四驱能力水平对置发动机没有吹嘘好 1 -1
CVT好急加速不行 0 -1
水平对置发动机会烧机油 0 -1
十万公里前提店保险全时四驱发动机可靠 0 1
关注好几年森林新款下来换车没2.5儿子那试驾过几次感觉动力2.0差些 0 -1
楼主你好地方定下来发动机燃烧以后产生水蒸气尾段产生冷凝水正常采纳 0 1
动力有点吃力建议2.0rav4 0 -1
知丅动力高买2T朋友真的很少 0 1
发动机新居然海蜡 0 1
我装发动机差速器护板没有适合四代变速箱护板高速跑120没有风噪问题主要出去玩走烂路公路更加放心一点 0 1
这种情况周围看来十万变速箱电磁阀故障率高拆下来看阀是不是堵厉害里面渣子特多 0 -1
车性能稳定外观大气值得推荐 0 1
指南总比EA888烧机油漏机油发动机加双离合专坑中国神车途观